In [1]:
import numpy as np
import pandas as pd
import joblib
import datetime
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from hyperopt import hp, anneal, Trials, STATUS_OK
from hyperopt.fmin import fmin
from functools import partial


In [2]:
# Load the datasets
train_df = pd.read_excel("train_data.xlsx")
test_df = pd.read_excel("test_data.xlsx")

In [3]:
# Define feature matrix and target variable
X_train = train_df.drop(['P'], axis=1)
y_train = train_df['P']
X_test = test_df.drop(['P'], axis=1)
y_test = test_df['P']

In [10]:
def model_metrics(model, X, y):
    """Compute RMSE for model evaluation."""
    y_pred = model.predict(X)
    return np.sqrt(mean_squared_error(y, y_pred))

def bayes_fmin(X_train, X_test, y_train, y_test, eval_iters=100):
    """Hyperparameter optimization using Bayesian search."""
    def objective(params):
        model = AdaBoostRegressor(
            learning_rate=float(params['learning_rate']),
            n_estimators=int(params['n_estimators']),
            random_state=42
        )
        model.fit(X_train, y_train)
        loss = model_metrics(model, X_test, y_test)
        return {"loss": loss, "status": STATUS_OK}

    space = {
        'n_estimators': hp.quniform('n_estimators', 1, 200, 1),
        'learning_rate': hp.quniform('learning_rate', 0.001, 1, 0.01),
    }
    best_params = fmin(objective, space, algo=partial(anneal.suggest), max_evals=eval_iters, trials=Trials())
    best_params["n_estimators"] = int(best_params["n_estimators"])
    best_params["learning_rate"] = float(best_params["learning_rate"])
    return best_params

In [11]:
# Hyperparameter tuning
best_params = bayes_fmin(X_train, X_test, y_train, y_test, 500)
print("Best Hyperparameters:", best_params)

100%|██████████| 500/500 [01:48<00:00,  4.61trial/s, best loss: 53.88789680494569]
Best Hyperparameters: {'learning_rate': 0.49, 'n_estimators': 19}


In [14]:
# Train AdaBoost model
model = AdaBoostRegressor(**best_params, random_state=42)
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Model Evaluation
metrics = {
    "RMSE_Train": np.sqrt(mean_squared_error(y_train, y_train_pred)),
    "MAE_Train": mean_absolute_error(y_train, y_train_pred),
    "R2_Train": r2_score(y_train, y_train_pred),
    "RMSE_Test": np.sqrt(mean_squared_error(y_test, y_test_pred)),
    "MAE_Test": mean_absolute_error(y_test, y_test_pred),
    "R2_Test": r2_score(y_test, y_test_pred)
}

for key, value in metrics.items():
    print(f"{key}: {value:.4f}")

RMSE_Train: 56.9043
MAE_Train: 40.4131
R2_Train: 0.8917
RMSE_Test: 53.8879
MAE_Test: 39.2130
R2_Test: 0.8906
